In [1]:
from tkinter import *
from tkinter import ttk
import tkinter.font as tkFont

In [2]:
from movie import Movie

In [3]:
from parse import parse_imdb_data

In [4]:
movies = parse_imdb_data()

Failed to parse movie 1......0.7      12   8.3  "By Any Means" (????) 
Failed to parse movie 1..1221..1       8   5.5  "Das große RTL Special zum Film" (????) {Evan Allmächtig} 
Failed to parse movie 0...0.0.06      12   8.5  "Das große RTL Special zum Film" (????) {Harry Potter und der Orden des Phönix} 
Failed to parse movie 200.0000.4      36   6.3  "Das große RTL Special zum Film" (????) {Harry Potter und die Kammer des Schreckens} 
Failed to parse movie 1....5.3..       6   5.8  "Germside" (????) 
Failed to parse movie ......46..       5   7.6  "Germspesyal" (????) 
Failed to parse movie ......22.6       5   9.0  "TV Rijnmond sport" (????) {Huldiging Feyenoord KNVB Beker 2008} 
Failed to parse movie 0000000124    1201   8.9  Baiohazâdo 7 (????) (VG) 
Failed to parse movie 2........8       5   8.2  Cumming of Age Volume 1 (????) (V) 
Failed to parse movie ....2....8       5   9.0  Faust Fucker - Dr. Fist und seine heißen Stuten (????) (V) 
Failed to parse movie 4...0.00.4      43  

In [27]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>5000, m_))
m_=list(filter(lambda m: m.mean_rating>7.5, m_))
len(m_)

3151

In [28]:
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) \
        for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    if(data[0][0].isnumeric()): # FIX: doesn't work for float
        data = [(float(d[0]), d[1]) for d in data]
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, \
        int(not descending)))

In [29]:
root = Tk()
root.title("IMDB MOVIE RECOMMENDER SYSTEM")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
mainframe.columnconfigure(0, weight=1)
mainframe.rowconfigure(0, weight=1)

filterframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Filter")
filterframe.grid(column=0, row=0, sticky=(N, E))
filterframe['borderwidth'] = 2
filterframe['relief'] = 'sunken'

yearfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Year")
yearfilterframe.grid(column=0, row=0, sticky=(N, E))
yearfilterframe['borderwidth'] = 2
yearfilterframe['relief'] = 'sunken'
ttk.Label(yearfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minyear=IntVar()
minyear.set(1900)
ttk.Entry(yearfilterframe, textvariable=minyear).grid(column=1, row=0, sticky="E")
ttk.Label(yearfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxyear=IntVar()
maxyear.set(2017)
ttk.Entry(yearfilterframe, textvariable=maxyear).grid(column=1, row=1, sticky="E")


numratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
numratingfilterframe.grid(column=0, row=1, sticky=(N, E))
numratingfilterframe['borderwidth'] = 2
numratingfilterframe['relief'] = 'sunken'
ttk.Label(numratingfilterframe, text="MIN").grid(column=0, row=0, sticky="E")
minratingcount=IntVar()
ttk.Entry(numratingfilterframe, textvariable=minratingcount).grid(column=1, row=0, sticky="W")
ttk.Label(numratingfilterframe, text="MAX").grid(column=0, row=1, sticky="E")
maxratingcount=IntVar()
maxratingcount.set(int(1e5))
ttk.Entry(numratingfilterframe, textvariable=maxratingcount).grid(column=1, row=1, sticky="W")


meanratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
meanratingfilterframe.grid(column=0, row=2, sticky=(N, E))
meanratingfilterframe['borderwidth'] = 2
meanratingfilterframe['relief'] = 'sunken'
ttk.Label(meanratingfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minrating=DoubleVar()
minrating.set(0.0)
ttk.Entry(meanratingfilterframe, textvariable=minrating).grid(column=1, row=0, sticky="E")
ttk.Label(meanratingfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxrating=DoubleVar()
maxrating.set(10.0)
ttk.Entry(meanratingfilterframe, textvariable=maxrating).grid(column=1, row=1, sticky="E")

def filter_data(*args):
    pass

def reset_filters(*args):
    pass

buttonfilterframe = ttk.Frame(filterframe, padding="3 3 12 12")
buttonfilterframe.grid(column=0, row=3, sticky=(N, E))

ttk.Button(buttonfilterframe, text="Filter", command=filter_data).grid(column=0, row=0, sticky = "W")
ttk.Button(buttonfilterframe, text="Reset", command=reset_filters).grid(column=1, row=0, sticky = "E")

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=1900, to=2017 , showvalue=True, label="Minimum Year", command=filter_year)
#s.grid(column=0, row=0, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0, to=1e3, resolution=1, showvalue=True, label="Minimum # Ratings", command=filter_num_ratings)
#s.grid(column=0, row=1, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0.0, to=10.0, resolution=0.1, showvalue=True, label="Minimum Mean Rating")
#s.grid(column=0, row=2, sticky=W)


dataframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Data")
dataframe.grid(column=1, row=0, sticky=(N, E))
dataframe['borderwidth'] = 2
dataframe['relief'] = 'sunken'

# some of the following code is borrowed from https://www.daniweb.com/programming/software-development/threads/350266/creating-table-in-python

data_header = ['Title', 'Year', 'Rating Count', "Mean Rating"]
#data = [('A', 1982, 5, 2.3), ['C', 2000, 3, 5]]



data = [[m.title, m.year, m.num_ratings, m.mean_rating] for m in m_]

data_tree = ttk.Treeview(dataframe, columns=data_header, show="headings")

vsb = ttk.Scrollbar(dataframe, orient="vertical", command=data_tree.yview)
hsb = ttk.Scrollbar(dataframe, orient="horizontal", command=data_tree.xview)

data_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

data_tree.grid(column=0, row=0, sticky='nsew')

vsb.grid(column=1, row=0, sticky='ns')
hsb.grid(column=0, row=1, sticky='ew')

dataframe.grid_columnconfigure(0, weight=1)
dataframe.grid_rowconfigure(0, weight=1)


for col in data_header:
    data_tree.heading(col, text=col.title(), command=lambda c=col: sortby(data_tree, c, 0))
    # adjust the column's width to the header string
    data_tree.column(col, width=tkFont.Font().measure(col.title()))
    
for item in data:
            data_tree.insert('', 'end', values=item)
            # adjust column's width if necessary to fit each value
            for ix, val in enumerate(item):
                col_w = tkFont.Font().measure(val)
                if data_tree.column(data_header[ix],width=None)<col_w:
                    data_tree.column(data_header[ix], width=col_w)

root.mainloop()

7603

In [15]:
movies[65000]

5 O'Clock in the Morning	1966	.2122.11..	9	4.4

In [60]:
"3".isdecimal()

True

In [59]:
"3.2".isdecimal()

False

In [61]:
"3".isnumeric()

True

In [62]:
"3.2".isnumeric()

False

In [63]:
"3".isdigit()

True

In [64]:
"3.2".isdigit()

False